# Handoff Orchestration

Handoff orchestration allows agents to transfer control to one another based on the context or user request. Each agent can “handoff” the conversation to another agent with the appropriate expertise, ensuring that the right agent handles each part of the task. This is particularly useful in customer support, expert systems, or any scenario requiring dynamic delegation.

* [Handoff Orchestration](https://learn.microsoft.com/en-us/semantic-kernel/frameworks/agent/agent-orchestration/handoff?pivots=programming-language-python)
* [Original Code](https://github.com/microsoft/semantic-kernel/blob/main/python/samples/getting_started_with_agents/multi_agent_orchestration/step4_handoff.py)

## Sample Description 

The following sample demonstrates how to create a handoff orchestration that represents a customer support triage system. The orchestration consists of 4 agents, each specialized in a different area of customer support: triage, refunds, order status, and order returns.

Depending on the customer's request, agents can hand off the conversation to the appropriate agent.

Human in the loop is achieved via a callback function similar to the one used in group chat orchestration. Except that in the handoff orchestration, all agents have access to the human response function, whereas in the group chat orchestration, only the manager has access to the human response function.

This sample demonstrates the basic steps of creating and starting a runtime, creating a handoff orchestration, invoking the orchestration, and finally waiting for the results.

In [1]:
# Libraries
from dotenv import load_dotenv
import os
import asyncio

from semantic_kernel.agents import Agent, ChatCompletionAgent, HandoffOrchestration, OrchestrationHandoffs
from semantic_kernel.agents.runtime import InProcessRuntime
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.contents import AuthorRole, ChatMessageContent, FunctionCallContent, FunctionResultContent
from semantic_kernel.functions import kernel_function

In [2]:
#load variables
load_dotenv()

# Variables - Azure Services
azopenai_ep=os.environ["AZURE_OPENAI_ACCOUNT"]
azopenai_key=os.environ["AZURE_OPENAI_KEY"]
azopenai_model=os.environ["AZURE_OPENAI_MODEL"]

### Create an instance of AzureChatCompletion with the provided configuration

In [3]:
az_chat_completion = AzureChatCompletion(
    deployment_name=azopenai_model,
    api_key=azopenai_key,
    endpoint=azopenai_ep 
)

### Plugins

We need to define the plugins that will be used in the agents. These plugins will contain the logic for handling specific tasks

In [4]:
class OrderStatusPlugin:
    @kernel_function
    def check_order_status(self, order_id: str) -> str:
        """Check the status of an order."""
        # Simulate checking the order status
        return f"Order {order_id} is shipped and will arrive in 2-3 days."

class OrderRefundPlugin:
    @kernel_function
    def process_refund(self, order_id: str, reason: str) -> str:
        """Process a refund for an order."""
        # Simulate processing a refund
        print(f"Processing refund for order {order_id} due to: {reason}")
        return f"Refund for order {order_id} has been processed successfully."

class OrderReturnPlugin:
    @kernel_function
    def process_return(self, order_id: str, reason: str) -> str:
        """Process a return for an order."""
        # Simulate processing a return
        print(f"Processing return for order {order_id} due to: {reason}")
        return f"Return for order {order_id} has been processed successfully."


### Agents and HandOff

* First Part: Define the agents that will use these plugins.
* Second Part: Use OrchestrationHandoffs to specify which agent can hand off to which, and under what circumstances.

In [5]:
def get_agents() -> tuple[list[Agent], OrchestrationHandoffs]:
    """
    Returns a tuple containing a list of customer support agents and their handoff relationships.

    The agents include:
    - TriageAgent: Handles general customer requests and triages issues.
    - RefundAgent: Handles refund-related requests.
    - OrderStatusAgent: Handles order status inquiries.
    - OrderReturnAgent: Handles order return requests.

    The OrchestrationHandoffs object defines which agents can hand off conversations to others based on the type of customer issue.
    """
    
    support_agent = ChatCompletionAgent(
        name="TriageAgent",
        description="A customer support agent that triages issues.",
        instructions="Handle customer requests.",
        service=az_chat_completion,
    )

    refund_agent = ChatCompletionAgent(
        name="RefundAgent",
        description="A customer support agent that handles refunds.",
        instructions="Handle refund requests.",
        service=az_chat_completion,
        plugins=[OrderRefundPlugin()],
    )

    order_status_agent = ChatCompletionAgent(
        name="OrderStatusAgent",
        description="A customer support agent that checks order status.",
        instructions="Handle order status requests.",
        service=az_chat_completion,
        plugins=[OrderStatusPlugin()],
    )

    order_return_agent = ChatCompletionAgent(
        name="OrderReturnAgent",
        description="A customer support agent that handles order returns.",
        instructions="Handle order return requests.",
        service=az_chat_completion,
        plugins=[OrderReturnPlugin()],
    )
    
    # Define the handoff relationships between agents
    handoffs = (
        OrchestrationHandoffs()
        .add_many(
            source_agent=support_agent.name,
            target_agents={
                refund_agent.name: "Transfer to this agent if the issue is refund related",
                order_status_agent.name: "Transfer to this agent if the issue is order status related",
                order_return_agent.name: "Transfer to this agent if the issue is order return related",
            },
        )
        .add(
            source_agent=refund_agent.name,
            target_agent=support_agent.name,
            description="Transfer to this agent if the issue is not refund related",
        )
        .add(
            source_agent=order_status_agent.name,
            target_agent=support_agent.name,
            description="Transfer to this agent if the issue is not order status related",
        )
        .add(
            source_agent=order_return_agent.name,
            target_agent=support_agent.name,
            description="Transfer to this agent if the issue is not order return related",
        )
    )

    return [support_agent, refund_agent, order_status_agent, order_return_agent], handoffs

### Observe Agent Responses

You can define a callback to print each agent's message as the conversation progresses.

In [6]:
def agent_response_callback(message: ChatMessageContent) -> None:
    """Observer function to print the messages from the agents.

    Please note that this function is called whenever the agent generates a response,
    including the internal processing messages (such as tool calls) that are not visible
    to other agents in the orchestration.
    """
    print(f"{message.name}: {message.content}")
    for item in message.items:
        if isinstance(item, FunctionCallContent):
            print(f"Calling '{item.name}' with arguments '{item.arguments}'")
        if isinstance(item, FunctionResultContent):
            print(f"Result from '{item.name}' is '{item.result}'")

### Human in the Loop

A key feature of handoff orchestration is the ability for a human to participate in the conversation. This is achieved by providing a human_response_function callback, which is called whenever an agent needs input from the user.

In [7]:
def human_response_function() -> ChatMessageContent:
    """Observer function to print the messages from the agents."""
    user_input = input("User: ")
    return ChatMessageContent(role=AuthorRole.USER, content=user_input)

### Set Up the Handoff Orchestration

Create a HandoffOrchestration object, passing in the agents, handoff relationships, and the callbacks.

In [13]:
# 1. Create a handoff orchestration with multiple agents
agents, handoffs = get_agents()
handoff_orchestration = HandoffOrchestration(
    members=agents,
    handoffs=handoffs,
    agent_response_callback=agent_response_callback,
    human_response_function=human_response_function,
)

In [14]:
# 2. Create a runtime and start it
runtime = InProcessRuntime()
runtime.start()

TriageAgent: Hello! Thank you for reaching out to support. How can I assist you today?
User: 

In [15]:
# 3. Invoke the orchestration with a task and the runtime
orchestration_result = await handoff_orchestration.invoke(
    task="Greet the customer who is reaching out for support.",
    runtime=runtime,
)

In [11]:
# 4. Wait for the results
value = await orchestration_result.get()
print(value)

Task is completed with summary: Greeted the customer reaching out for support but no further requests were provided.


In [12]:
# 5. Stop the runtime after the invocation is complete
await runtime.stop_when_idle()